In [18]:
import torch
import torch.nn.functional as F
from torch.distributions import constraints

import pyro
import pyro.distributions as dist
from pyro.infer import Predictive, SVI, Trace_ELBO
from pyro.optim import Adam

from gpytorch.kernels import RBFKernel, ScaleKernel
import gpytorch.distributions as gdist

import pdb
from pmextract import extract

In [19]:
G = 5 # max number of functions
A = 7 # max number of arguments per function
func_alpha = 1.0 # concentration parameter for function DP
markov_alpha = 1.0 # concentration parameter for how old your args should be

In [20]:
data = torch.ones(3, 9)

In [21]:
class GPKernel(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.kernel = ScaleKernel(RBFKernel())
    def forward(self, input_locs):
        return self.kernel(input_locs)

In [22]:
kernel = GPKernel()

In [23]:
def mix_weights(beta):
    "Turn iid Beta samples into the weights of a categorical (Stick Breaking DP)"
    beta1m_cumprod = (1 - beta).cumprod(-1)
    return F.pad(beta, (0, 1), value=1) * F.pad(beta1m_cumprod, (1, 0), value=1)

In [62]:
def partition(num_funcs, func_ids, outputs, inputs):
    return [(inputs[func_ids == i], outputs[func_ids == i]) for i in range(num_funcs)]

In [72]:
def model(data):
    T, M = data.shape # timesteps by variables
    N = T * M # total number of variables
    pyro.module("kernel", kernel)
    mu = pyro.param("mu", torch.randn(G))
    with pyro.plate("func_betas", G -1):
        func_beta = pyro.sample("func_beta", dist.Beta(1, func_alpha))
    with pyro.plate("function", N):
        from_function = pyro.sample("from_function", dist.Categorical(mix_weights(func_beta)))
        num_funcs = from_function.max() + 1
    with pyro.plate("arg_betas", T - 1):
        arg_beta = pyro.sample("arg_beta", dist.Beta(1, markov_alpha))
    with pyro.plate("func_arg_dists", num_funcs):
        func_arg_params = pyro.sample("func_arg_weights", dist.Dirichlet(torch.ones(M)))
    time_weights = torch.stack([F.pad(mix_weights(arg_beta[:i]), (0, T - i - 1)) for i in range(T)])
    with pyro.plate("arguments", N) as varindex:
        t = varindex.div(M, rounding_mode='trunc')
        with pyro.plate("nth_arg", A):
            arg_times = pyro.sample("arg_times", dist.Categorical(time_weights[t]))
            arg_vars = pyro.sample("arg_vars", dist.Categorical(func_arg_params[from_function]))
    pairs = partition(num_funcs, from_function, data.view(-1), data[arg_times, arg_vars].T)
    for i, pair in enumerate(pairs):
        gp_in, gp_out = pair
        k_xx = kernel(gp_in)
        pyro.sample(f"data_{i}", gdist.MultivariateNormal(mu[i:i+1].expand(k_xx.shape[0]), k_xx), obs=gp_out)

In [73]:
model(data)

/home/sam/.local/lib/python3.9/site-packages/pyro/primitives.py:138: RuntimeWarning: trying to observe a value outside of inference at data_0
  warnings.warn(


TODO: pick up from here. Debug this!

Shouldn't there be 18 variables? Why are there 23?

In [48]:
k_xx.shape

torch.Size([18, 18])

In [49]:
gp_out.shape

torch.Size([18])

In [31]:
pdb.pm()

> /tmp/ipykernel_641901/2010763094.py(24)model()
     20             arg_vars = pyro.sample("arg_vars", dist.Categorical(func_arg_params[from_function]))
     21         gp_ins, gp_outs = partition(num_funcs, from_function, data.view(-1), data[arg_times, arg_vars].T)
     22         for gp_in, gp_out in zip(gp_ins, gp_outs):
     23             k_xx = kernel(gp_in)
---> 24             pyro.sample("data", gdist.MultivariateNormal(mu.expand(k_xx.shape[0]), k_xx), obs=gp_out)

ipdb> extract()
*** RuntimeError: Copied <module>'s variables to <module>
ipdb> q


In [27]:
pdb.pm()

> /tmp/ipykernel_641901/947917112.py(4)partition()
      2     # gp_out is a list (size n-funcs) of vectors (size 'examples')
      3     # gp_in is a list (size n-funcs) of (examples x n-args) matrices
----> 4     gp_out = [inputs[func_ids == i] for i in range(func_ids)]
      5     gp_in = [outputs[func_ids == i] for i in range(func_ids)]
      6     return gp_in, gp_out

ipdb> func_ids.dtype
torch.int64
ipdb> type(inputs)
<class 'torch.Tensor'>
ipdb> inputs[func_ids == i]
*** NameError: name 'i' is not defined
ipdb> inputs[func_ids == 2]
tensor([[1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1.]])
ipdb> q


In [102]:
data[arg_times, arg_vars].shape # which argument x which variable. 

torch.Size([7, 27])

## GP Stuff

 Start with vanilla GPs. Eventually add fancy stuff like Kroncker structure or SKIP or variational versions.

Do we do this variationally, or exactly? We could do it exactly. Finding the log determinant of the covariance matrix will be the part that could take a long time. We could try both. 

To do it exactly, just make the covariance matrix, pass it to gpytorch MultivariateNormal, and you're good to go

## Other Thoughts

Idea: Bayesian programming learning in general gets you bounds on the causal effect size at the same time as it produces plausable programs. Because we know the true program is in the posterior somewhere, so if we just sample effects from the posterior at the same time as we sample parameters, we get a bound. 

Specifically, if we want to know a bound on causal effect size, we can check the differences between observations of $B_t$ in the original and observations of $B_t$ under intervention. That is, to check the causal effect in a given model, sample $\frac{1}{N}\sum_{i=1}^N B_t - B_t'$ from it.

What are the interventions here? With binary variables, that's easy. Just set or not set the variable of a variable. 

Alternately: there's also the perspective that causal effect MEANS that there's a directed path between the variables. We could just record, for each model sampled, whether such a path exists. This will tell you the posterior probability that A causes B. 